In [1]:
# reference: dive into deep learning - ch06
import d2lzh as d2l
import math
from mxnet import autograd, nd
from mxnet.gluon import loss as gloss
import time

In [7]:
( corpus_indices, char_to_idx, idx_to_char, vocab_size ) = d2l.load_data_jay_lyrics()

In [10]:
def to_onehot(X, size):
    return [nd.one_hot(x,size) for x in X.T]

In [11]:
X = nd.arange(10).reshape((2,5))
inputs = to_onehot(X, vocab_size)
len(inputs), inputs[0].shape

(5, (2, 1027))

In [62]:
vocab_size

1027

In [63]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = d2l.try_gpu()
print('will use', ctx)

will use cpu(0)


In [88]:

def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01,shape=shape,ctx=ctx)
    #隐藏层参数
    W_xh = _one((num_inputs,num_hiddens))
    W_hh = _one((num_hiddens,num_hiddens))
    b_h = nd.zeros(num_hiddens,ctx=ctx)
    #输出层参数
    W_hq = _one((num_hiddens,num_outputs))
    b_q = nd.zeros(num_outputs,ctx=ctx)
    #附上梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    #print (W_xh.shape)
    for param in params:
        param.attach_grad()
    return params

In [89]:
def init_rnn_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size,num_hiddens),ctx=ctx),)

In [85]:
def rnn(inputs,state,params):
    # inputs和outputs皆为num_steps个形状为（batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        #print (W_xh.shape)
        #H = nd.tanh(nd.dot(X,W_hh)+ nd.dot(H,W_hh)+b_h)
        #print (nd.dot(H,W_hh).shape)
        H = nd.tanh(nd.dot(X,W_xh)+nd.dot(H,W_hh)+b_h)
        Y = nd.dot(H,W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [90]:
state = init_rnn_state(X.shape[0],num_hiddens,ctx)
inputs = to_onehot(X.as_in_context(ctx), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
len(outputs), outputs[0].shape, state_new[0].shape

(5, (2, 1027), (2, 256))

In [97]:
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
               num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, ctx)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars):
        # 将上一时间步的输出作为当前时间步的输入
        X = to_onehot(nd.array([output[-1]],ctx=ctx), vocab_size)
        # 计算输出和更新隐藏状态
        (Y,state) = rnn(X,state,params)
        # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t+1]])
        else: 
            output.append(int(Y[0].argmax(axis=1).asscalar()))
    return ''.join([idx_to_char[i] for i in output])
    

In [101]:
predict_rnn('分开',10,rnn,params, init_rnn_state, num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx)

'分开带C刚护潮美用必非'

In [116]:
def grad_clipping(params, theta, ctx):
    norm = nd.array([0],ctx)
    for param in params:
        norm += (param.grad**2).sum()
    norm = norm.sqrt().asscalar()
    if norm > theta:
        for param in params:
            param.grad[:] *= theta/norm

In [112]:
def train_and_predict_rnn(rnn,get_params, init_rnn_stats, num_hiddens,
                         vocab_size, ctx, corpus_indices, idx_to_char,
                         char_to_idx, is_random_iter, num_epochs, num_steps, 
                         lr, clipping_theta, batch_size, pred_period, 
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
        
    params = get_params()
    loss = gloss.SoftmaxCrossEntropyLoss()
    
    for epoch in range(num_epochs):
        if not is_random_iter: #如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, ctx)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, ctx)
        for X, Y in data_iter:
            if is_random_iter: # 如果使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, ctx)
            else: # 否则需要使用detach函数计算图分离隐藏状态
                for s in state:
                    s.detach()
            with autograd.record():
                inputs = to_onehot(X, vocab_size)
                # outputs有num_steps个形状为（batch_size, vocab_size)的矩阵
                (outputs, state) = rnn(inputs, state, params)
                # 连结之后形状为(num_steps*batch_size, vocab_size)
                outputs = nd.concat(*outputs, dim=0)
                # Y的形状是(batch_size, num_steps),转置之后再变成长度为
                # batch*num_steps的向量，这样跟输出的行一一对应
                y = Y.T.reshape((-1,))
                # 使用交叉熵损失计算平均分类误差
                l = loss(outputs, y).mean()
            l.backward()
            grad_clipping(params, clipping_theta, ctx)
            d2l.sgd(params, lr, 1) #因为误差已经取过均值， 梯度不用再做平均
            l_sum += l.asscalar()*y.size
            n += y.size
            
        if (epoch+1)%pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch+1, math.exp(l_sum/n), time.time()-start)
                 )
            for prefix in prefixes:
                print(' -', predict_rnn(
                    prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx))

In [104]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [117]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                     vocab_size, ctx, corpus_indices, idx_to_char,
                     char_to_idx, True, num_epochs, num_steps, lr,
                     clipping_theta, batch_size, pred_period, pred_len,
                     prefixes)

epoch 50, perplexity 66.667743, time 1.07 sec
 - 分开 我不要再想爱 你知哈 我给我的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏
 - 不分开 快子我 别怪我 我不要 我爱我的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 
epoch 100, perplexity 9.401083, time 1.06 sec
 - 分开 一直用双 在谁村外 在一场梦 你一定纵 不果海中 你一定空 不颗海中 你在一定 你的手空 不果海
 - 不分开亚 我爱你这 我想了好生活 后知后觉 我该好好生活 我知道好生活 不要不觉 你已经离开我 不知不
epoch 150, perplexity 2.839450, time 1.07 sec
 - 分开 不是再双天打我妈妈 我说的话 你甘已在太记 不知后觉 我该好好生活 我该得很生活 不知不觉 你已
 - 不分开吗 我叫你爸想 我不要再想 我不 我不 我不能再想你 爱情不是 你已经离开我 不知不觉 我跟了这
epoch 200, perplexity 1.576397, time 1.09 sec
 - 分开 一愿心双 他的完容 一切海动 没有种空 象一场梦 不敢去碰 没有再 痛不知珍重 也有苦衷 问候了
 - 不分开扫 我不能再想 我不 我不 我不能 爱情走的太快就像龙卷风 不能承受我已无处可躲 我不要再想 我
epoch 250, perplexity 1.298585, time 1.15 sec
 - 分开 出默心空 他谁中容 盲都有从 疗伤止痛 不再感动 没有梦 痛不知轻重 一只灰狼 问候怕日出 白色
 - 不分开期 然后将过去 慢慢温习 让我爱上你 那场悲剧 是你完美演出的一场戏 宁愿心碎哭泣 再狠狠忘记 


In [118]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                     vocab_size, ctx, corpus_indices, idx_to_char,
                     char_to_idx, False, num_epochs, num_steps, lr,
                     clipping_theta, batch_size, pred_period, pred_len,
                     prefixes)

epoch 50, perplexity 61.711115, time 1.05 sec
 - 分开 我想要这 你有了空 我有你这 我有你这 我有你这 我有你这 我有你这 我有你这 我有你这 我有你
 - 不分开 我想要你 你有了空 我有你这 我有你这 我有你这 我有你这 我有你这 我有你这 我有你这 我有
epoch 100, perplexity 7.252342, time 1.08 sec
 - 分开 一颗的 旧时光的脚头 一地是停医 你将它开抽抢 它果入秋 漫天黄沙凉过 哼哼哈兮 快果黄沙截过 
 - 不分开觉 你天经离 我跟种发 你么出痛 没有没痛 没不放纵 没有没有 没有己纵 恨不己痛 没有没痛 没
epoch 150, perplexity 2.127260, time 1.05 sec
 - 分开 一候了 旧有了中我留 有话啊 是不是你不想活 说你怎么面对我 甩开球我满腔的怒火 我想揍你已经很
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一 秋惯夜空 你想的梦 你在出纵 恨一
epoch 200, perplexity 1.298990, time 1.05 sec
 - 分开 一候我 谁是神枪手 巫师盘在旧自我也妈 我是的话 你甘会听 不要再这样打我妈妈 难道你手不会痛吗
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 后知了一个秋 后知后觉 我该好好生活 我该好
epoch 250, perplexity 1.170554, time 1.03 sec
 - 分开 问候我 说怪神枪手 巫师 他念念 有词的 对酋长下诅咒  说骷用着我妈妈愿恼 我不要再你经再提起
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好
